In [4]:
import numpy as np
from pathlib import Path
from collections import defaultdict as ddict
from collections import OrderedDict as odict
from easydict import EasyDict as edict
import pandas as pd

### 1. transform dk data with new labels

In [3]:
def labNew2Old(string):
    prefix = "ctx-"
    itm1, itm2 = string.split(".")
    return prefix + itm1 + "-" + itm2

In [6]:
filDir = Path("../data/brain_plot_meg/")

fcROld = pd.read_csv(filDir/'DK_dictionary_normalized.csv').set_index('Unnamed: 0')
dkLabs = pd.read_excel(filDir/'DK_labs_list.xlsx')
fcRarr = np.array(fcROld)

oldLabs = list(fcROld.columns)
dkLabs = list(np.array(dkLabs).reshape(-1))

In [25]:
fcRNewDic = odict()

for lab in dkLabs:
    oldlab = labNew2Old(lab)
    fcRNewDic[oldlab] = list(fcROld[oldlab])
    
fcRNew = pd.DataFrame(fcRNewDic)
fcRNew.index = fcROld.index
fcRNew.to_csv(filDir/"DK_dic68.csv")

### 2. Obtain the U, weighted U and corresponding correlation for fMRI data

In [12]:
import sys
sys.path.append("/home/huaqingj/MyResearch/TVDN")
from pyTVDN.tmpUtils import *
import pickle
import scipy.stats as ss

#### Get the fcR

In [5]:
fcRDF = pd.read_csv("../necessary files/AALICA.csv")
fcR = np.array(fcRDF)[:90, :]
names = list(fcRDF.columns)
names = [name.strip() for name in names]

fcR = np.array([minmax(fcR[:, i]) for i in range(7)])
fcR = fcR.T

#### U 

In [14]:
datDir = Path("../results/fMRIHPFs_fixnchg5")
fil = list(datDir.glob("*MinVar.pkl"))[0]
print(fil)

with open(fil, "rb") as f:
    TVDNres = pickle.load(f)
    
Us = []
for DetObj in TVDNres.DetObjs:
    r = DetObj.paras.r
    rUs = DetObj.midRes.eigVecs[:, :r]
    kpidxs = np.where(np.diff(np.concatenate([[np.Inf], np.abs(rUs[0, :])])) != 0)[0]
    for ix in kpidxs:
        Us.append(rUs[:, ix])
        
absUs = np.array(np.abs(Us))
absUsList =[minmax(absUs[i, :]) for i in range(absUs.shape[0])]
absUs = np.array(absUsList)

../results/fMRIHPFs_fixnchg5/fMRIResultMinVar.pkl


#### correlation

In [18]:
corrs = ddict(list)
for absU in absUs:
    for j, name in enumerate(names):
        corrs[name].append(ss.pearsonr(fcR[:, j], absU)[0])
        
corDF = pd.DataFrame(corrs)
corArr = np.array(corDF)

np.savetxt(datDir/"allCorrU.txt", corArr)